In [ ]:
import duckdb

con = duckdb.connect()
# manifest with duplicate Cxxxx codes.
con.execute('''
             CREATE VIEW unique_files AS 
             SELECT file 
             FROM read_csv_auto("../data/cache/bls/files.csv") 
             GROUP BY file 
             ORDER BY file 
             ''')

print(con.sql('''
SELECT COUNT(*) AS file_count
FROM unique_files
              ''').df())
con.close()

In [ ]:
import duckdb
con = duckdb.connect('../data/analysis.duckdb')

con.execute('''
        CREATE OR REPLACE TABLE missing_codes AS 
        WITH per_code AS (
            SELECT
            Code::BIGINT AS Code,
            COUNT(*) AS years_present,
            SUM(CASE WHEN quarter_count = 4 THEN 1 ELSE 0 END) AS full_years,
            SUM(quarter_count) AS total_quarters
            FROM bls_manifest_summary
            WHERE Year BETWEEN 2014 AND 2024
            GROUP BY Code
        )
        SELECT p.*, d.Area, d.Title
        FROM per_code p
        JOIN dim_metro_full d USING (Code)
        ORDER BY full_years DESC, years_present DESC;
''')
print(con.sql('describe missing_codes').df())
print(con.sql('''
        select Area, Code, Title  from missing_codes
        where years_present = full_years              
        
''').df())
#print(con.sql("select * from missing_codes").df())
con.close()

In [ ]:
import duckdb

con = duckdb.connect()
con.execute('''
             CREATE VIEW manifest AS
             SELECT 
             SUBSTRING(Code, 2, 5) || '0' AS Code, 
             Year, 
             SUBSTRING(Qtrcsv, 1, 1) AS Qtr 
             FROM read_csv_auto("../data/cache/bls/manifest.csv", header=True, sep="_")
             ''')
con.execute("ATTACH '../data/analysis.duckdb' AS analysis_db")
con.execute('''
        CREATE OR REPLACE TABLE analysis_db.bls_manifest_summary AS     
        SELECT 
            Code, 
            Year, 
            Count(Qtr) AS quarter_count, 
            STRING_AGG(Qtr, ', ') AS quarters
        FROM manifest
        GROUP BY Code, Year
        ''')

print(con.sql("select * from analysis_db.bls_manifest_summary").df())
con.close()

In [ ]:
import duckdb

con = duckdb.connect("../data/derived/annual_wages.parquet")


In [ ]:
print(con.sql('select table_name, column_name, data_type from information_schema."columns";').df())


In [ ]:
df = con.sql("SELECT * FROM annual_wages;").df()
print(df.head())

In [ ]:
con.close()